In [2]:
import sys
!{sys.executable} -m pip3 install lightgbm

/usr/local/bin/python3: No module named pip3


In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import sklearn
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("../assets/GOOGLE.csv")
df 

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,2.490664,2.591785,2.390042,2.499133,2.499133,897427216
1,2004-08-20,2.515820,2.716817,2.503118,2.697639,2.697639,458857488
2,2004-08-23,2.758411,2.826406,2.716070,2.724787,2.724787,366857939
3,2004-08-24,2.770615,2.779581,2.579581,2.611960,2.611960,306396159
4,2004-08-25,2.614201,2.689918,2.587302,2.640104,2.640104,184645512
...,...,...,...,...,...,...,...
4912,2024-02-26,143.449997,143.839996,138.740005,138.750000,138.750000,33513000
4913,2024-02-27,139.410004,140.490005,138.500000,140.100006,140.100006,22364000
4914,2024-02-28,139.100006,139.279999,136.639999,137.429993,137.429993,30628700
4915,2024-02-29,138.350006,139.949997,137.570007,139.779999,139.779999,35447100


In [150]:
x = list(df.drop(['Close', 'Date', 'Adj Close'], axis=1).columns)
y = ['Close']
x

['Open', 'High', 'Low', 'Volume']

In [152]:
train=df.sample(frac=0.8,random_state=200)
predict=df.drop(train.index)
predict[x]

,Open,High,Low,Volume
2,2.758411,2.826406,2.716070,366857939
17,2.676219,2.789544,2.659780,217608605
18,2.753679,2.845086,2.744712,215279909
19,2.798012,2.884189,2.780827,186207345
37,3.348449,3.427403,3.322546,234419380
...,...,...,...,...
4884,143.429993,145.839996,143.056000,19198900
4893,153.639999,155.199997,152.919998,20909300
4897,140.889999,143.880005,138.169998,42116900
4903,149.539993,150.589996,148.559998,17236100


In [138]:
new_train = lgb.Dataset(train[x], train[y])
new_predict = lgb.Dataset(predict[x], predict[y])

In [139]:
params = {
    "boosting_type": "gbdt",
    "num_leaves": 5,
    "force_row_wise": True,
    "learning_rate": 0.5,
    "metric": "binary_logloss",
    "bagging_fraction": 0.8,
    "feature_fraction": 0.8,
    "min_data_in_leaf": 15,
    "use_quantized_grad": True,
    "num_grad_quant_bins": 4,
    "quant_train_renew_leaf": True,

}

Model = lgb.train(params, new_train, num_boost_round = 100)

[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 3934, number of used features: 4
[LightGBM] [Info] Start training from score 43.084753


In [140]:
y_pred = Model.predict(predict[x])

In [141]:
result = pd.concat([pd.DataFrame(y_pred), predict[y].reset_index().drop('index', axis =1)], axis=1, ignore_index=True)

In [142]:
result = result.set_axis(['y', 'y_pred'], axis = 1)
result

,y,y_pred
0,2.664719,2.724787
1,2.678888,2.776842
2,2.678888,2.789544
3,2.678888,2.838610
4,3.227821,3.422173
...,...,...
978,145.356091,144.080002
979,150.036001,154.839996
980,140.287476,143.539993
981,149.212465,148.729996


In [143]:
result.corr()

,y,y_pred
y,1.000000,0.999829
y_pred,0.999829,1.000000
